# ONNX

## Convert YOLO Model To ONNX

In [ ]:
!pip install ultralytics

In [2]:
from ultralytics import YOLO

model = YOLO("yolov8l.pt")

In [3]:
model.export(format="onnx")

Ultralytics YOLOv8.2.84 🚀 Python-3.10.12 torch-2.4.0+cu121 CPU (Intel Xeon 2.20GHz)
YOLOv8l summary (fused): 268 layers, 43,668,288 parameters, 0 gradients, 165.2 GFLOPs

PyTorch: starting from 'yolov8l.pt' with input shape (1, 3, 640, 640) BCHW and output shape(s) (1, 84, 8400) (83.7 MB)

ONNX: starting export with onnx 1.16.2 opset 19...
ONNX: export success ✅ 6.5s, saved as 'yolov8l.onnx' (166.8 MB)

Export complete (16.7s)
Results saved to /content
Predict:         yolo predict task=detect model=yolov8l.onnx imgsz=640  
Validate:        yolo val task=detect model=yolov8l.onnx imgsz=640 data=coco.yaml  
Visualize:       https://netron.app


'yolov8l.onnx'

## Convert Sklearn Model To ONNX

In [ ]:
!pip install skl2onnx

In [5]:
import skl2onnx
from sklearn.datasets import load_digits
from sklearn.model_selection import train_test_split
from sklearn.neighbors import KNeighborsClassifier
import numpy as np

In [6]:
dataset = load_digits()
X = dataset.images.reshape(len(dataset.images),-1)
Y = dataset.target
x_train , x_test , y_train , y_test = train_test_split(X,Y,test_size=0.2)

In [7]:
model = KNeighborsClassifier(n_neighbors=7)
model.fit(x_train,y_train)

KNeighborsClassifier(n_neighbors=7)

In [8]:
onnx_model = skl2onnx.to_onnx(model, x_train[0])

with open("sklearn_to_onnx_model.onnx", "wb") as f:
  f.write(onnx_model.SerializeToString())

## Convert Torch Model To ONNX

In [ ]:
!pip install onnx
!pip install onnxscript

In [10]:
import torch
import torchvision
import onnx

In [11]:
class Model(torch.nn.Module):
    def __init__(self):
        super().__init__()
        self.fully_connect1 = torch.nn.Linear(784,512)
        self.fully_connect2 = torch.nn.Linear(512,256)
        self.fully_connect3 = torch.nn.Linear(256,10)

    def forward(self,x):
        x = x.reshape((x.shape[0],784))
        x = self.fully_connect1(x)
        x = torch.relu(x)
        x = torch.dropout(x,0.2,train=True)
        x = self.fully_connect2(x)
        x = torch.relu(x)
        x = torch.dropout(x,0.5,train=True)
        x = self.fully_connect3(x)
        x = torch.softmax(x,dim=1)

        return x

model = Model()

In [12]:
input_shape = torch.randn(1, 1, 28, 28)
onnx_model = torch.onnx.dynamo_export(model, input_shape)
onnx_model.save("pytorch_to_onnx_model.onnx")

In [13]:
onnx_model = onnx.load_model("/content/pytorch_to_onnx_model.onnx")
onnx.checker.check_model(onnx_model)

## Convert TensorFlow Model To ONNX

In [ ]:
!pip install tf2onnx

In [15]:
import tensorflow as tf
from tensorflow.keras.applications import ResNet50
import tf2onnx

In [16]:
model = ResNet50(weights="imagenet", include_top=False)

In [17]:
input_shape = [tf.TensorSpec(model.inputs[0].shape, dtype=model.inputs[0].dtype, name=model.inputs[0].name)]
onnx_model, _ = tf2onnx.convert.from_keras(model, input_signature=input_shape)
onnx.save(onnx_model, "tensorflow_to_onnx_model.onnx")